In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import math
from ipywidgets import interactive
import ipywidgets as widgets

# for dark theme aficionados
plt.style.use('dark_background')

## Problem description
We propose to solve the 1D Helmholtz equation written in spherical coordinates. The problem is specified by 
- the physical domain $r \in [R_0,R]$
- a Neumann boundary condition at $r=R_0>0$
- a Robin absorbing boundary condition at $r=R$

The complex valued acoustic pressure $u(x)$ in the physical domain is governed by the following scalar Helmholtz equation:

$$ \frac{d^2u}{dr^2} + \frac{2}{r} \frac{du}{dr} + \left(k^2 - \frac{\ell(\ell+1)}{r^2}\right) u = 0, \; \text{in} \; \Omega $$

where $k=\omega/c_0$ is the acosutic wavenumber, and $c_0$ the speed of sound. For a fixed value $\ell \in \mathbb{N}$, the analytical solution ($-\imath \omega t$ convention) is given by the spherical Hankel function

$$ u_{\text{ex}}(r) = A_{\ell} h^{(1)}_{\ell} (kr) $$

The boundary condition at $r=R_0$ defines the value of $A_{\ell}$ through the relation

$$ \left.\begin{matrix} \frac{d u_{\text{ex}}}{d r} \end{matrix}\right|_{r=R_0} = A_{\ell} k \left( -h^{(1)}_{\ell+1} (kR_0)  + \frac{\ell}{k R_0} h^{(1)}_{\ell} (kR_0) \right) $$

We will solve the problem thanks to a continuous Galerkin method, which leads after integrating by parts to the variational problem:

Find $u \in H^1(\Omega)$ such that:

$$ \forall v \in H^1(\Omega), \quad \int_{R_0}^R \left\{ \frac{d u}{d r} \overline{\frac{d v}{d r}} - \frac{2}{r} \frac{d u}{d r} \overline{v} - \left(k^2 - \frac{\ell(\ell+1)}{r^2}\right) u \overline{v} \right\} \, \text{d}r  = \left[ \frac{\partial u}{\partial n} \overline{v} \right]_{R_0}^R$$



Let us initiate the numerical model

In [2]:
from ExternalFunctions import Mesh1D, CreateDofs
def initiate(DuctLength, NrOfElem, Order, R0):
    NrOfNodes, Coord, Element = Mesh1D(DuctLength, NrOfElem, R0)
    NrOfDofs, DofNode, DofElement = CreateDofs(NrOfNodes,NrOfElem,Element,Order)
    return NrOfNodes, Coord, Element, NrOfDofs, DofNode, DofElement

In [3]:
def assemble_matrix(NrOfDofs,NrOfElem,Order,Coord,Element,DofElement,k,l):
    from ExternalFunctions import MassAndStiffness_spherical_1D
    Matrix = np.zeros((NrOfDofs,NrOfDofs), dtype=np.complex128)
    for iElem in np.arange(0,NrOfElem):
        # call the function returning the mass and stifness element matrices
        Ke, Ce, Me, Me_r2 = MassAndStiffness_spherical_1D(iElem, Order, Coord, Element)
        ElemDofs = (DofElement[:,iElem]).astype(int)
        # assemble
        Matrix[np.ix_(ElemDofs,ElemDofs)] += Ke - 2*Ce - k**2*Me + (l*(l+1))*Me_r2
    return Matrix

In [4]:
def assemble_impedance(Matrix,DofNode,NrOfNodes,k,R0,DuctLength):
    # now apply impedance boundary condition at last node (BGT-1)
    Matrix[DofNode[NrOfNodes-1],DofNode[NrOfNodes-1]] += - ( 1j*k - 1/(2*(R0 + DuctLength)) )
    return Matrix
	
def assemble_rhs(NrOfDofs,DofNode,k,R0,l,Al):
    from AnalyticalSolutions import d_spherical_hankel
    # and the velocity at first node
    Rhs = np.zeros((NrOfDofs,1), dtype=np.complex128)
    Rhs[DofNode[0]] = -Al*k*d_spherical_hankel(l,k*R0)
    return Rhs

In [5]:
def solve(Matrix, Rhs):
    # solve the sparse system of equations 
    Sol = np.linalg.solve(Matrix, Rhs) 
    return Sol

We now compute the solution

In [6]:
def compute_model(k, Order, NrOfElem, DuctLength, l, R0=0.5, Al=1.):
    from ExternalFunctions import GetSolutionOnSubgrid 
    from AnalyticalSolutions import spherical_hankel
    h = DuctLength/NrOfElem # mesh size
    d_lambda = 2*math.pi/(k*h)*Order # nr of dofs per wavelength
    # first create the mesh and the Dofs list
    NrOfNodes, Coord, Element, NrOfDofs, DofNode, DofElement = initiate(DuctLength,NrOfElem,Order,R0)
    # then assemble the matrix
    Matrix = assemble_matrix(NrOfDofs,NrOfElem,Order,Coord,Element,DofElement,k,l)
    # now apply impedance boundary condition at last node
    Matrix = assemble_impedance(Matrix,DofNode,NrOfNodes,k,R0,DuctLength)
    # assemble RHS
    Rhs = assemble_rhs(NrOfDofs,DofNode,k,R0,l,Al)
    # solve the sparse system of equations 
    Sol = solve(Matrix,Rhs)
    # compute the solution on a subgrid 
    Lambda = 2*math.pi/k; NrOfWavesOnDomain = DuctLength/Lambda
    x_sub, u_h_sub = GetSolutionOnSubgrid(Sol, Order, Coord, Element, NrOfElem, DofElement, NrOfWavesOnDomain)
    # exact solution on subgrid 
    u_exact_sub = Al*spherical_hankel(l,k*x_sub)
    return x_sub, u_h_sub, u_exact_sub, NrOfDofs, d_lambda, Sol, Coord, Element

Plotting feature

In [7]:
def plot_result(x_sub, u_h_sub, u_exact_sub, NrOfDofs, d_lambda, NrOfElem, Coord, Element, Order):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4), tight_layout=True)
    ax.plot(x_sub, np.real(u_h_sub), label="$Re(u_h)$",linewidth=2)
    ax.plot(x_sub, np.real(u_exact_sub), label="$Re(u_{exact})$",linewidth=2) 
    ax.legend(loc="best",fontsize=16)
    ax.set(xlabel=r'$x$', ylabel=r'real part',
           title='NrOfDofs = '+str(NrOfDofs) + ', ($d_\lambda$=%1.4g' %d_lambda +' Dofs per wavelength)')
    plt.show()

def compute_interactive(k, Order, NrOfElem, DuctLength, l):
    x_sub, u_h_sub, u_exact_sub, NrOfDofs, d_lambda, Sol, Coord, Element = compute_model(k, Order, NrOfElem, DuctLength, l)
    plot_result(x_sub, u_h_sub, u_exact_sub, NrOfDofs, d_lambda, NrOfElem, Coord, Element, Order)
	# error in the L2 norm
    E2 = np.linalg.norm(u_h_sub - u_exact_sub)/np.linalg.norm(u_exact_sub)*100
    print('-' *100 +'\n'+' Numerical error (L2-norm) is %1.4g' %E2 + ' % \n' +'-' *100)

In [8]:
# parameters of the problem
# R0: input radius
# l: spatial mode
# Al: modal amplitude of the analytical solution
# k: wavenumber

interactive_plot = interactive(compute_interactive, k=widgets.IntSlider(value=10, min=1, max=50, description='k'),
                               Order=widgets.IntSlider(value=3, min=1, max=8, description='Order'),
                               NrOfElem=widgets.IntSlider(value=20, min=1, max=200, description='Nb of Elts'),
                               DuctLength=widgets.IntSlider(value=1, min=1, max=5, description='Duct length'),
							   l=widgets.IntSlider(value=0, min=0, max=30, description='Harmonic'))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=10, description='k', max=50, min=1), IntSlider(value=3, description='Ord…